In [16]:
import re
import json
import string
with open('galaxy.json', encoding='utf-8') as f:
    data = json.load(f)

In [17]:
import nltk
from nltk.tokenize import word_tokenize

In [18]:
nltk.download('stopwords')
nltk.download('punkt')
punct = string.punctuation + "—" + "«" + "»"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\redic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\redic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
from nltk.corpus import stopwords
stopwords_list = stopwords.words("russian")

Очищстим текст от пунктуации, чисел и стоп-слов, приведем все слова к нижнему регистру

In [20]:
for i in range(len(data)):
    data[str(i)]['text'] = re.sub(r'[0-9]+', ' ', data[str(i)]['text'])
    data[str(i)]['text'] = re.sub(r"\b([а-яА-Я]\.)",' ', data[str(i)]['text'])
    tokens = word_tokenize(data[str(i)]['text'])    
    tokens = [token.lower() for token in tokens if token not in punct]
    data[str(i)]['text'] = [word for word in tokens if word not in stopwords_list]

Создаем новый  словарь, а также записываем его в новый файл json

In [21]:
lst = []
for i in range(len(data)):
    lst.append(data[str(i)]['text'])

In [22]:
data_new = {}
for i in range(len(lst)):
    data_new[str(i)] = {'words': lst[i]}

In [23]:
with open('fl_e.json', 'w', encoding='utf-8') as file:
    json.dump(data_new, file, indent=4, ensure_ascii=False)

Функция

In [24]:
import jellyfish
import numpy as np

In [25]:
def word_jel(word):
    ids = []
    for i in range(len(data_new)):
        if word in data_new[str(i)]['words']:
            ids.append(int(i)) 
    if len(ids) != 0:
        return ids
    for i in range(len(data_new)):
        if word not in data_new[str(i)]['words']:
            for w in data_new[str(i)]['words']:
                ids.append(jellyfish.levenshtein_distance(word, w))
    x = min(int(s) for s in ids)
    ids.clear()
    for i in range(len(data_new)):
        for w in data_new[str(i)]['words']:
            if jellyfish.levenshtein_distance(word, w) == x:
                    ids.append(w) 
    uids = np.unique(ids)
    return uids

In [26]:
word_jel('юрист')

array(['крест', 'рис', 'рост', 'сист', 'эрнст', 'юри'], dtype='<U5')

Тексты были взяты из Википедии, поэтому предположительно опечаток нет. Попробуем проверить.

In [27]:
def check_spelling(text):
    domain = "https://speller.yandex.net/services/spellservice.json"
    words = text.split()
    if len(words) == 1:
        request = requests.get(domain + "/checkText?text=" + words[0])
        if requests:
            if not request.json():
                return None
            else:
                return request.json()[0]["word"], request.json()[0]["s"]
        else:
            return None
    elif len(words) > 1:
        words = "+".join(words)
        request = requests.get(domain + "/checkText?text=" + words)
        if requests:
            if not request.json():
                words = words.split("+")
                return None
            else:
                response = [(i["word"], i["s"]) for i in request.json()]
                return response
        else:
            return None
    return None


In [28]:
import requests
for i in range(len(data_new)):
    lst_2 =  data_new[str(i)]['words']
    for word in lst_2:
        stri = ''
        stri += word + ' '
        check = check_spelling(stri)
        if check_spelling(stri) is not None:
            for i in range(len(check[1])):
                if len(re.findall(r'[^а-яА-Я]', check[1][i] )) == 0:
                    new_word = check[1][i]
                    break
                else:
                    continue
            if stri in data_new[str(i)]['words']:
                stri in data_new[str(i)]['words'] == new_word

In [30]:
lst_new = []
for i in range(len(data_new)):
    lst_new.append(data_new[str(i)]['words'])

In [31]:
import json
data_ag = {}
for i in range(len(lst_new)):
    data_ag[i] = {'words': lst_new[i]}

In [32]:
with open('galaxy_words.json', 'w', encoding='utf-8') as file:
    json.dump(data_ag, file, indent=4, ensure_ascii=False)